# Module 6 Lab

This lab uses the eleven Month-XX.csv files provided via the class Canvas module page.

# Exercise 1

Iterate over each of the Month-XX.csv files using a list comprehension or for loop and (i) import the data as a
DataFrame and (ii) compute the total number of observations in each DataFrame. Which DataFrame has the
most observations? Which has the least?

In [1]:
import os
import glob
import zipfile
import pandas as pd
import random

In [3]:
import glob

csv_files = sorted(
    glob.glob(r"C:\Users\Gokul\Downloads\Month-XX-data\data\*.csv")
)

print(f"Found {len(csv_files)} files:")
for f in csv_files:
    print(f)


Found 11 files:
C:\Users\Gokul\Downloads\Month-XX-data\data\Month-01.csv
C:\Users\Gokul\Downloads\Month-XX-data\data\Month-02.csv
C:\Users\Gokul\Downloads\Month-XX-data\data\Month-03.csv
C:\Users\Gokul\Downloads\Month-XX-data\data\Month-04.csv
C:\Users\Gokul\Downloads\Month-XX-data\data\Month-05.csv
C:\Users\Gokul\Downloads\Month-XX-data\data\Month-06.csv
C:\Users\Gokul\Downloads\Month-XX-data\data\Month-07.csv
C:\Users\Gokul\Downloads\Month-XX-data\data\Month-08.csv
C:\Users\Gokul\Downloads\Month-XX-data\data\Month-09.csv
C:\Users\Gokul\Downloads\Month-XX-data\data\Month-10.csv
C:\Users\Gokul\Downloads\Month-XX-data\data\Month-11.csv


In [4]:
# Exercise 1: import each file and compute number of observations
dfs = {os.path.basename(f): pd.read_csv(f) for f in csv_files}
counts = {name: df.shape[0] for name, df in dfs.items()}

print("Exercise 1: observation counts per file")
for name in sorted(counts):
    print(f"{name}: {counts[name]}")
most = max(counts.items(), key=lambda x: x[1])
least = min(counts.items(), key=lambda x: x[1])
print(f"\nMost observations: {most[0]} with {most[1]} rows")
print(f"Least observations: {least[0]} with {least[1]} rows")

Exercise 1: observation counts per file
Month-01.csv: 54535
Month-02.csv: 44380
Month-03.csv: 53259
Month-04.csv: 51033
Month-05.csv: 55079
Month-06.csv: 59666
Month-07.csv: 64268
Month-08.csv: 69492
Month-09.csv: 71855
Month-10.csv: 80277
Month-11.csv: 94315

Most observations: Month-11.csv with 94315 rows
Least observations: Month-02.csv with 44380 rows


# Exercise 2

Import the Month-XX.csv files into your current Python session. Do so using a for loop or list
comprehension. Rather than have 11 separate data frames (one for each month), combine these so that you
have one data frame containing all the data. Your final data frame should have 698,159 rows and 10 columns.
What is the Account_ID value for the very last transaction in this combined DataFrame? Note, the last
transaction in the combined DataFrame should be the last transaction in the Month-11.csv data set.

In [8]:
combined_df = pd.concat([dfs[f"Month-{str(i).zfill(2)}.csv"] for i in range(1, 12)],
                        ignore_index=True)

print("Combined shape:", combined_df.shape)

last_account_id = combined_df.iloc[-1]["Account_ID"]
print("Account_ID of last transaction:", last_account_id)


Combined shape: (698159, 10)
Account_ID of last transaction: 495150


# Exercise 3

How many unique values exist in each column?

In [9]:
combined_df.nunique()

Account_ID               475413
Transaction_Timestamp    686538
Factor_A                      7
Factor_B                      6
Factor_C                      4
Factor_D                     15
Factor_E                     62
Response                     42
Transaction_Status            2
Month                        11
dtype: int64

# Exercise 4

Create a function convert_to_qtr() that converts monthly values to quarters. This function should assess
a month value (“Jan”, “Feb”, ... , “Dec”) and convert to “Q1”, “Q2”, “Q3”, or “Q4”. Do it such that:

If the month input is Jan-Mar, then the function returns “Q1”

If the month input is Apr-Jun, then the function returns “Q2”

If the month input is Jul-Sep, then the function returns “Q3”

If the month input is Oct-Dec, then the function returns “Q4”

Apply this function to the imported data (note the Month column contains month values in the form of “Jan”,
“Feb”, ... , “Dec”) and create a new column that contains the quarter that each observation is aligned to.
Compute how many observations fall into each quarter

In [13]:
def convert_to_qtr(month_str):
    if not isinstance(month_str, str):
        return None
    m = month_str.strip()[:3].title()  
    q1 = {"Jan","Feb","Mar"}
    q2 = {"Apr","May","Jun"}
    q3 = {"Jul","Aug","Sep"}
    q4 = {"Oct","Nov","Dec"}
    if m in q1:
        return "Q1"
    if m in q2:
        return "Q2"
    if m in q3:
        return "Q3"
    if m in q4:
        return "Q4"
    return None

combined_df["Quarter"] = combined_df["Month"].apply(convert_to_qtr)

quarter_counts = combined_df["Quarter"].value_counts().reindex(["Q1","Q2","Q3","Q4"]).fillna(0).astype(int)

print("Observations per quarter:")
print(quarter_counts)

Observations per quarter:
Q1    152174
Q2    165778
Q3    205615
Q4    174592
Name: Quarter, dtype: int32


# Exercise 5

Import the Month-XX.csv files into your current Python session. Do so using a for loop or list
comprehension. However, this time I want you to only import those data sets that have 60,000 or more
observations. You'll need to include a conditional statement within your for loop or list comprehension to
make this happen. Once you've imported the data sets that meet this condition combine them into one single
DataFrame. How many observations does this DataFrame have?

In [14]:
dfs_over_60k = [
    df
    for name, df in dfs.items()
    if df.shape[0] >= 60000
]

combined_over_60k = pd.concat(dfs_over_60k, ignore_index=True)

print("Number of observations:", combined_over_60k.shape[0])

Number of observations: 380207


# Exercise 6

Create a function ( get_data() ) that performs the task in exercise 5. However, this function should have two
arguments:
1. files which you can use to feed the list of file names to import and
2. min_req_obs which you can specify the number of observations required in a data set for it to be
imported.
The output of the function should be a single DataFrame containing all observations from those data sets that
meet the required observation threshold. Consequently, you should be able to use this function as below to get
the same combined DataFrame that you got in exercise 5:
get_data(files, min_req_obs=60000)
Now use this function to import all Month-XX.csv files that contain at least 75,000 observations and combine
them into one single DataFrame. How many observations is in this DataFrame?

In [20]:
def get_data(dfs_dict, min_req_obs):
    
    selected = [
        df
        for name, df in dfs_dict.items()
        if df.shape[0] >= min_req_obs
    ]
    
    if not selected:
        return pd.DataFrame()
    
    return pd.concat(selected, ignore_index=True)

In [21]:
combined_75k = get_data(dfs, min_req_obs=75000)
print("Number of observations:", combined_75k.shape[0])

Number of observations: 174592


# Exercise 7
Now use the get_data() function to iterate over the following values for min_req_obs : 25000, 50000,
75000, 90000. How many observations are in the final DataFrames based on these min_req_obs values?

In [22]:
def get_data(dfs_dict, min_req_obs):
    selected = [
        df
        for name, df in dfs_dict.items()
        if df.shape[0] >= min_req_obs
    ]
    if not selected:
        return pd.DataFrame()
    return pd.concat(selected, ignore_index=True)


In [23]:
thresholds = [25000, 50000, 75000, 90000]
results = {}
for t in thresholds:
    df_t = get_data(dfs, min_req_obs=t)
    results[t] = df_t.shape[0]

print("Observations for each threshold:")
for t, count in results.items():
    print(f"min_req_obs = {t}: {count} observations")

Observations for each threshold:
min_req_obs = 25000: 698159 observations
min_req_obs = 50000: 653779 observations
min_req_obs = 75000: 174592 observations
min_req_obs = 90000: 94315 observations


# Exercise 8

For this exercise and the ones that follow you will not be using any external data sets.
Create a function divisible(a, b) that accepts two integers ( a and b ) and returns True if a is
divisble by b without a remainder. For example, divisible(10, 3) should return False , while
divisible(6, 3) should return True .
Once you have created this function, apply it to the randomly generated integers below. Be sure to use the
same seed value. What is your result?
import random
random.seed(123)
a = random.randint(10, 100)
b = random.randint(1, 10)
divisible(a, b)

In [25]:
import random
def divisible(a, b):
    return a % b == 0

random.seed(123)
a = random.randint(10, 100)
b = random.randint(1, 10)

result = divisible(a, b)

print("a =", a)
print("b =", b)
print("divisible(a, b) =", result)

a = 16
b = 5
divisible(a, b) = False


# Exercise 9
Create a function lucky_sum() that takes all the integers a user enters and returns their sum. However, if one
of the values is 13 then it does not count towards the sum, nor do any values to its right.
For example, your function should behave as follows:
lucky_sum(1, 2, 3, 4)
10
lucky_sum(1, 13, 3, 4)
1
lucky_sum(1, 3, 13, 4)
4
lucky_sum(13)
0
Once you've created this function, apply it to the following randomly sampled list of numbers. Be sure to run the
code below exactly as you see it. What is your result?
random.seed(18)
my_values = random.choices(range(1, 14), k=26)
lucky_sum(*my_values)

In [28]:
import random

def lucky_sum(*nums):
    total = 0
    for n in nums:
        if n == 13:
            break
        total += n
    return total

random.seed(18)
my_values = random.choices(range(1, 14), k=26)

result = lucky_sum(*my_values)

print("Values:", my_values)
print("lucky_sum result:", result)

Values: [3, 9, 5, 3, 7, 7, 7, 6, 4, 4, 9, 5, 9, 11, 12, 13, 4, 4, 10, 8, 3, 9, 12, 11, 3, 4]
lucky_sum result: 101
